# Türkçe BERT Embedding Fine-Tuning - Soru Cevap Seti

Bu notebook, `dbmdz/bert-base-turkish-cased` modelini soru-cevap çiftleri için fine-tune eder.

## Kurulum

In [ ]:
!pip install sentence-transformers transformers datasets accelerate torch -q

## Kütüphaneleri İçe Aktarma

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import numpy as np
from sklearn.model_selection import train_test_split
import json

# GPU kontrolü
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Kullanılan cihaz: {device}")

## 1. Veri Yükleme

Verinizin formatı şu şekilde olmalı:
- CSV: 'soru' ve 'cevap' sütunları
- JSON: [{"soru": "...", "cevap": "..."}, ...]
- Excel: 'soru' ve 'cevap' sütunları

In [ ]:
# Veri yükleme fonksiyonu
def load_qa_data(file_path):
    """
    Soru-cevap verilerini yükler.
    Desteklenen formatlar: CSV, JSON, Excel
    """
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
    elif file_path.endswith('.json'):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
    elif file_path.endswith(('.xlsx', '.xls')):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Desteklenmeyen dosya formatı!")
    
    # Sütun isimlerini kontrol et ve standartlaştır
    if 'soru' not in df.columns or 'cevap' not in df.columns:
        print(f"Mevcut sütunlar: {df.columns.tolist()}")
        raise ValueError("Veri setinde 'soru' ve 'cevap' sütunları olmalı!")
    
    return df

# ÖRNEK: Eğer elinizde veri yoksa, örnek veri oluşturabilirsiniz:
# Kendi veri dosyanızın yolunu yazın
DATA_FILE = 'soru_cevap_veri.csv'  # Buraya kendi dosya yolunuzu yazın

# Örnek veri oluşturma (test için)
sample_data = {
    'soru': [
        'Python nedir?',
        'Makine öğrenmesi ne demektir?',
        'Deep learning nedir?',
        'BERT modeli nasıl çalışır?',
        'Transformer mimarisi nedir?'
    ],
    'cevap': [
        'Python, yüksek seviyeli, yorumlamalı bir programlama dilidir.',
        'Makine öğrenmesi, bilgisayarların verilerden öğrenmesini sağlayan yapay zeka dalıdır.',
        'Deep learning, çok katmanlı yapay sinir ağları kullanarak öğrenen bir makine öğrenmesi yöntemidir.',
        'BERT, çift yönlü transformer mimarisini kullanan bir doğal dil işleme modelidir.',
        'Transformer, dikkat mekanizması kullanan modern bir sinir ağı mimarisidir.'
    ]
}

# Örnek veri kaydetme (gerçek verileriniz varsa bu bölümü atlayın)
df_sample = pd.DataFrame(sample_data)
df_sample.to_csv('soru_cevap_veri.csv', index=False, encoding='utf-8')

# Veriyi yükle
try:
    df = load_qa_data(DATA_FILE)
    print(f"✓ Toplam {len(df)} soru-cevap çifti yüklendi")
    print("\nİlk birkaç örnek:")
    print(df.head())
except Exception as e:
    print(f"Hata: {e}")
    print("Lütfen veri dosyanızın yolunu ve formatını kontrol edin.")

## 2. Veri Ön İşleme

In [ ]:
# Boş değerleri temizle
df = df.dropna(subset=['soru', 'cevap'])

# Boşlukları temizle
df['soru'] = df['soru'].str.strip()
df['cevap'] = df['cevap'].str.strip()

# Çok kısa verileri filtrele (opsiyonel)
df = df[(df['soru'].str.len() > 5) & (df['cevap'].str.len() > 10)]

print(f"Temizleme sonrası: {len(df)} soru-cevap çifti")

# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(f"\nEğitim seti: {len(train_df)} örnekleri")
print(f"Test seti: {len(test_df)} örnekleri")

## 3. Model ve Tokenizer Yükleme

In [ ]:
# Türkçe BERT modelini yükle
model_name = 'dbmdz/bert-base-turkish-cased'

print(f"Model yükleniyor: {model_name}")
model = SentenceTransformer(model_name)
print("✓ Model başarıyla yüklendi")

# Model bilgileri
print(f"\nEmbedding boyutu: {model.get_sentence_embedding_dimension()}")
print(f"Max token uzunluğu: {model.max_seq_length}")

## 4. Training Data Hazırlama

Soru-cevap çiftlerini InputExample formatına dönüştürüyoruz.

In [ ]:
# Training örneklerini oluştur
train_examples = []
for idx, row in train_df.iterrows():
    # Her soru-cevap çifti için pozitif örnek oluştur (benzerlik skoru = 1.0)
    train_examples.append(InputExample(texts=[row['soru'], row['cevap']], label=1.0))

print(f"✓ {len(train_examples)} eğitim örneği oluşturuldu")

# DataLoader oluştur
train_batch_size = 16  # GPU belleğinize göre ayarlayın
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)

print(f"Batch sayısı: {len(train_dataloader)}")

## 5. Loss Fonksiyonu Seçimi

Soru-cevap embeddingleri için uygun loss fonksiyonları:
- **CosineSimilarityLoss**: Soru ve cevap arasındaki kosinüs benzerliğini öğrenir
- **MultipleNegativesRankingLoss**: Daha güçlü, negatif örnekler kullanır

In [ ]:
# Loss fonksiyonu seçimi
# Seçenek 1: Cosine Similarity Loss (Basit ve etkili)
train_loss = losses.CosineSimilarityLoss(model)

# Seçenek 2: Multiple Negatives Ranking Loss (Daha güçlü, önerilir)
# train_loss = losses.MultipleNegativesRankingLoss(model)

print(f"✓ Loss fonksiyonu: {type(train_loss).__name__}")

## 6. Evaluator Oluşturma

In [ ]:
# Test seti için evaluator
test_questions = test_df['soru'].tolist()
test_answers = test_df['cevap'].tolist()
test_scores = [1.0] * len(test_df)  # Tüm soru-cevap çiftleri pozitif

evaluator = evaluation.EmbeddingSimilarityEvaluator(
    test_questions, 
    test_answers, 
    test_scores,
    name='qa-test'
)

print("✓ Evaluator oluşturuldu")

## 7. Model Fine-Tuning

In [ ]:
# Training parametreleri
num_epochs = 4  # Epoch sayısı
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)  # %10 warmup
output_path = './turkish-bert-qa-finetuned'  # Model kaydedilecek yer

print("="*50)
print("FINE-TUNING BAŞLIYOR")
print("="*50)
print(f"Epoch sayısı: {num_epochs}")
print(f"Batch size: {train_batch_size}")
print(f"Warmup steps: {warmup_steps}")
print(f"Model kaydedilecek yer: {output_path}")
print("="*50)

# Fine-tuning başlat
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=output_path,
    evaluation_steps=500,  # Her 500 adımda bir değerlendirme
    save_best_model=True,
    show_progress_bar=True
)

print("\n✓ Fine-tuning tamamlandı!")
print(f"Model kaydedildi: {output_path}")

## 8. Model Test Etme

In [ ]:
# Fine-tune edilmiş modeli yükle
finetuned_model = SentenceTransformer(output_path)

# Test sorusu
test_soru = "Yapay zeka nedir?"

# Soru embeddingini al
soru_embedding = finetuned_model.encode(test_soru, convert_to_tensor=True)

# Tüm cevapların embeddinglerini al
cevap_embeddings = finetuned_model.encode(df['cevap'].tolist(), convert_to_tensor=True)

# Cosine similarity hesapla
from sentence_transformers.util import cos_sim

cosine_scores = cos_sim(soru_embedding, cevap_embeddings)[0]

# En yüksek skorlu cevapları bul
top_k = min(5, len(df))
top_results = torch.topk(cosine_scores, k=top_k)

print(f"\nSoru: {test_soru}")
print("\nEn benzer cevaplar:")
print("="*80)

for idx, (score, index) in enumerate(zip(top_results[0], top_results[1]), 1):
    print(f"\n{idx}. Benzerlik Skoru: {score:.4f}")
    print(f"   Soru: {df.iloc[index]['soru']}")
    print(f"   Cevap: {df.iloc[index]['cevap']}")

## 9. Model Performans Karşılaştırması

In [ ]:
# Orijinal vs Fine-tuned model karşılaştırması
original_model = SentenceTransformer(model_name)

def test_model_performance(model, name):
    """Model performansını test eder"""
    print(f"\n{name} Model Performansı:")
    print("="*50)
    
    # Test seti üzerinde ortalama benzerlik skoru hesapla
    similarities = []
    
    for idx, row in test_df.head(10).iterrows():  # İlk 10 test örneği
        soru_emb = model.encode(row['soru'], convert_to_tensor=True)
        cevap_emb = model.encode(row['cevap'], convert_to_tensor=True)
        similarity = cos_sim(soru_emb, cevap_emb).item()
        similarities.append(similarity)
        print(f"Soru-Cevap {idx+1} benzerliği: {similarity:.4f}")
    
    print(f"\nOrtalama benzerlik: {np.mean(similarities):.4f}")
    return np.mean(similarities)

# Karşılaştırma
original_score = test_model_performance(original_model, "Orijinal")
finetuned_score = test_model_performance(finetuned_model, "Fine-tuned")

print("\n" + "="*50)
print("SONUÇ:")
print(f"Orijinal model skoru: {original_score:.4f}")
print(f"Fine-tuned model skoru: {finetuned_score:.4f}")
print(f"İyileşme: {((finetuned_score - original_score) / original_score * 100):.2f}%")
print("="*50)

## 10. Modeli Kullanma - Örnek Fonksiyon

In [ ]:
def find_best_answer(question, knowledge_base_df, model, top_k=3):
    """
    Verilen soru için en iyi cevabı bulur.
    
    Args:
        question: Kullanıcı sorusu
        knowledge_base_df: Soru-cevap dataframe'i
        model: Fine-tuned model
        top_k: Gösterilecek en iyi K sonuç
    
    Returns:
        En iyi cevaplar ve skorlar
    """
    # Soru embeddingini al
    question_embedding = model.encode(question, convert_to_tensor=True)
    
    # Tüm cevapların embeddinglerini al
    answer_embeddings = model.encode(
        knowledge_base_df['cevap'].tolist(), 
        convert_to_tensor=True
    )
    
    # Benzerlik hesapla
    scores = cos_sim(question_embedding, answer_embeddings)[0]
    
    # En iyi sonuçları bul
    top_results = torch.topk(scores, k=min(top_k, len(knowledge_base_df)))
    
    results = []
    for score, idx in zip(top_results[0], top_results[1]):
        results.append({
            'soru': knowledge_base_df.iloc[idx]['soru'],
            'cevap': knowledge_base_df.iloc[idx]['cevap'],
            'skor': score.item()
        })
    
    return results

# Örnek kullanım
test_questions = [
    "Programlama dilleri nelerdir?",
    "Derin öğrenme nasıl çalışır?",
    "Yapay zeka uygulamaları nelerdir?"
]

for q in test_questions:
    print(f"\n{'='*80}")
    print(f"Soru: {q}")
    print(f"{'='*80}")
    
    results = find_best_answer(q, df, finetuned_model, top_k=2)
    
    for i, result in enumerate(results, 1):
        print(f"\n{i}. Sonuç (Skor: {result['skor']:.4f})")
        print(f"   Benzer Soru: {result['soru']}")
        print(f"   Cevap: {result['cevap']}")

## 11. Model ve Tokenizer'ı Kaydetme

In [ ]:
# Model zaten output_path'e kaydedildi
# Ek olarak Hugging Face Hub'a yüklemek isterseniz:

# from huggingface_hub import login
# login()  # Token ile giriş yapın
# finetuned_model.save_to_hub("kullanici-adi/turkish-bert-qa-finetuned")

print(f"\n✓ Model başarıyla kaydedildi: {output_path}")
print("\nModeli yüklemek için:")
print(f"model = SentenceTransformer('{output_path}')")

## Notlar ve İpuçları

### Hyperparameter Tuning:
- **Batch size**: GPU belleğinize göre 8, 16, 32
- **Epochs**: Genellikle 2-5 epoch yeterli
- **Learning rate**: Varsayılan genellikle iyidir (2e-5)
- **Warmup steps**: Total steps'in %10'u önerilir

### Loss Fonksiyonları:
- **CosineSimilarityLoss**: Basit ve hızlı, soru-cevap çiftleri için iyi
- **MultipleNegativesRankingLoss**: Daha güçlü, büyük veri setleri için önerilir
- **ContrastiveLoss**: Negatif örneklerle çalışır
- **TripletLoss**: Anchor-positive-negative üçlüleri kullanır

### Veri Artırma:
- Aynı anlama gelen farklı soru formları ekleyin
- Negatif örnekler (ilgisiz soru-cevap çiftleri) ekleyin
- Veri setinizi büyütmek için paraphrase teknikleri kullanın

### Performans İyileştirme:
- Daha fazla veri toplayın
- Veri kalitesini artırın
- Farklı loss fonksiyonları deneyin
- Hyperparameter tuning yapın